# VQE Tutorial: Deep Dive into Variational Quantum Eigensolver

This notebook provides an in-depth exploration of VQE for protein folding.

## Contents

1. VQE Theory and Algorithm
2. Hamiltonian Construction
3. Ansatz Design and Comparison
4. Optimizer Selection
5. Parameter Initialization Strategies
6. Noise Analysis
7. Convergence Analysis
8. Scaling Studies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time

from quantum_protein_folding.models import VQEFoldingModel
from quantum_protein_folding.data.loaders import load_hp_sequence
from quantum_protein_folding.data.preprocess import map_to_lattice
from quantum_protein_folding.quantum.hamiltonian import (
    build_hamiltonian,
    compute_exact_ground_state
)
from quantum_protein_folding.quantum.circuit_builder import (
    build_hardware_efficient_ansatz,
    count_circuit_resources
)
from quantum_protein_folding.analysis import plot_convergence, analyze_convergence

Path("../results").mkdir(exist_ok=True)
print("✓ Setup complete")

## 1. VQE Theory

VQE minimizes the expectation value:

$$E(\theta) = \langle \psi(\theta) | H | \psi(\theta) \rangle$$

where $|\psi(\theta)\rangle = U(\theta)|0\rangle^{\otimes n}$ is a parameterized ansatz.

### Protein Folding Hamiltonian

$$H = H_{\text{contact}} + \lambda H_{\text{backbone}} + \mu H_{\text{bias}}$$

- **Contact term**: Residue-residue interactions (MJ potentials)
- **Backbone term**: Connectivity and self-avoidance constraints
- **Bias term**: Compactness regularization

## 2. Hamiltonian Construction

In [ ]:
# Load sequence
sequence_str = "HPHPPHHPHH"
sequence = load_hp_sequence(sequence_str)

# Create lattice encoding
encoding = map_to_lattice(
    sequence,
    lattice_dim=2,
    encoding_type='turn_direction',
    constraint_weight=10.0,
    bias_weight=0.1
)

hamiltonian = encoding.hamiltonian

print(f"Hamiltonian Information:")
print(f"  Number of qubits: {hamiltonian.num_qubits}")
print(f"  Number of terms: {len(hamiltonian)}")
print(f"  Encoding: {encoding.encoding_type}")
print(f"\nFirst 5 Pauli terms:")
for pauli, coeff in list(hamiltonian.to_list())[:5]:
    print(f"  {pauli}: {coeff:.4f}")

### Exact Ground State (for small systems)

In [ ]:
if hamiltonian.num_qubits <= 12:  # Only for small systems
    print("Computing exact ground state via diagonalization...")
    ground_energy, ground_state = compute_exact_ground_state(hamiltonian)
    print(f"Exact ground state energy: {ground_energy:.6f}")
    print("This is the target energy VQE should approach.")
else:
    print(f"System too large ({hamiltonian.num_qubits} qubits) for exact diagonalization")

## 3. Ansatz Design Comparison

We'll compare different ansatz types and depths.

In [ ]:
# Test different ansatz configurations
ansatz_configs = [
    {'type': 'hardware_efficient', 'depth': 1},
    {'type': 'hardware_efficient', 'depth': 2},
    {'type': 'hardware_efficient', 'depth': 3},
]

print("Ansatz Circuit Resources:\n")
print(f"{'Type':<20} {'Depth':<8} {'#Qubits':<10} {'#Params':<10} {'Circuit Depth':<15} {'CNOT Count':<12}")
print("-" * 85)

for config in ansatz_configs:
    circuit, params = build_hardware_efficient_ansatz(
        n_qubits=encoding.n_qubits,
        depth=config['depth'],
        entanglement='linear'
    )
    
    resources = count_circuit_resources(circuit)
    
    print(f"{config['type']:<20} {config['depth']:<8} {resources['n_qubits']:<10} "
          f"{resources['n_parameters']:<10} {resources['depth']:<15} {resources['cx_count']:<12}")

print("\nNote: Deeper ansatz = more expressivity but harder optimization")

## 4. Optimizer Comparison

Different optimizers have different characteristics for VQE.

In [ ]:
optimizers = ['COBYLA', 'SLSQP', 'SPSA']
results = {}

for opt_name in optimizers:
    print(f"\n{'='*50}")
    print(f"Running VQE with {opt_name} optimizer...")
    print(f"{'='*50}")
    
    model = VQEFoldingModel(
        sequence=sequence_str,
        lattice_dim=2,
        ansatz_depth=2,
        optimizer=opt_name,
        shots=1024
    )
    
    start = time.time()
    result = model.run(maxiter=50)
    elapsed = time.time() - start
    
    results[opt_name] = {
        'energy': result.optimal_value,
        'time': elapsed,
        'iterations': result.n_iterations,
        'history': result.convergence_history
    }
    
    print(f"  Final energy: {result.optimal_value:.4f}")
    print(f"  Time: {elapsed:.2f}s")
    print(f"  Iterations: {result.n_iterations}")

### Compare Optimizer Performance

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot convergence curves
for opt_name, data in results.items():
    axes[0].plot(data['history'], label=opt_name, linewidth=2, alpha=0.8)

axes[0].set_xlabel('Iteration', fontsize=12)
axes[0].set_ylabel('Energy', fontsize=12)
axes[0].set_title('Optimizer Convergence Comparison', fontweight='bold', fontsize=13)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Bar chart of final energies
opt_names = list(results.keys())
energies = [results[name]['energy'] for name in opt_names]

axes[1].bar(opt_names, energies, alpha=0.7, edgecolor='black', linewidth=2)
axes[1].set_ylabel('Final Energy', fontsize=12)
axes[1].set_title('Final Energy Comparison', fontweight='bold', fontsize=13)
axes[1].grid(True, axis='y', alpha=0.3)

# Highlight best
best_idx = np.argmin(energies)
axes[1].patches[best_idx].set_color('green')
axes[1].patches[best_idx].set_alpha(0.9)

plt.tight_layout()
plt.savefig('../results/vqe_optimizer_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nBest optimizer: {opt_names[best_idx]} (E={energies[best_idx]:.4f})")

## 5. Parameter Initialization Strategies

Initial parameters can significantly affect convergence.

In [ ]:
# Create model
model = VQEFoldingModel(
    sequence=sequence_str,
    lattice_dim=2,
    ansatz_depth=2,
    optimizer='COBYLA'
)

n_params = model.solver.n_params

# Test different initialization strategies
init_strategies = {
    'Random [0, 2π]': 2 * np.pi * np.random.rand(n_params),
    'Small random': 0.1 * np.random.rand(n_params),
    'Zeros': np.zeros(n_params),
    'Uniform π/2': np.ones(n_params) * np.pi / 2,
}

init_results = {}

for name, init_params in init_strategies.items():
    print(f"Testing: {name}")
    
    model.solver.initial_params = init_params
    result = model.run(maxiter=50)
    
    init_results[name] = result.optimal_value
    print(f"  Final energy: {result.optimal_value:.4f}\n")

# Summary
print("\nInitialization Strategy Summary:")
for name, energy in sorted(init_results.items(), key=lambda x: x[1]):
    print(f"  {name:<20}: {energy:.4f}")

## 6. Noise Analysis (Optional)

For realistic simulations, we can add noise models.

In [ ]:
from qiskit_aer.noise import NoiseModel, depolarizing_error

# Create noise model
def create_noise_model(error_rate=0.01):
    noise_model = NoiseModel()
    
    # Depolarizing error on single-qubit gates
    single_qubit_error = depolarizing_error(error_rate, 1)
    noise_model.add_all_qubit_quantum_error(single_qubit_error, ['u1', 'u2', 'u3', 'rx', 'ry', 'rz'])
    
    # Depolarizing error on two-qubit gates
    two_qubit_error = depolarizing_error(error_rate * 2, 2)
    noise_model.add_all_qubit_quantum_error(two_qubit_error, ['cx', 'cz'])
    
    return noise_model

# Compare noiseless vs noisy
noise_levels = [0.0, 0.005, 0.01, 0.02]
noise_results = {}

for noise_level in noise_levels:
    print(f"Running with noise level: {noise_level:.3f}")
    
    if noise_level > 0:
        noise_model = create_noise_model(noise_level)
    else:
        noise_model = None
    
    # Note: Would need to pass noise_model to VQEFoldingModel
    # For now, just demonstrate the concept
    
    print(f"  (Noise simulation with error rate {noise_level})\n")

print("Note: Full noise simulation requires backend integration")

## 7. Convergence Analysis

In [ ]:
# Run VQE with more iterations
model = VQEFoldingModel(
    sequence=sequence_str,
    lattice_dim=2,
    ansatz_depth=3,
    optimizer='COBYLA'
)

result = model.run(maxiter=200)

# Analyze convergence
conv_analysis = analyze_convergence(result.convergence_history)

print("Convergence Analysis:")
for key, value in conv_analysis.items():
    print(f"  {key}: {value}")

# Plot with moving average
fig, ax = plt.subplots(figsize=(10, 6))

history = np.array(result.convergence_history)
iterations = np.arange(len(history))

# Raw data
ax.plot(iterations, history, 'b-', alpha=0.3, label='Raw')

# Moving average
window = 10
if len(history) >= window:
    moving_avg = np.convolve(history, np.ones(window)/window, mode='valid')
    ax.plot(iterations[window-1:], moving_avg, 'r-', linewidth=2, label=f'MA({window})')

ax.axhline(y=conv_analysis['best_value'], color='g', linestyle='--', label='Best')

ax.set_xlabel('Iteration', fontsize=12)
ax.set_ylabel('Energy', fontsize=12)
ax.set_title('VQE Convergence Analysis', fontweight='bold', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/vqe_convergence_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Scaling Study

How does VQE performance scale with protein length?

In [ ]:
# Test different sequence lengths
test_sequences = [
    "HPHP",          # 4 residues
    "HPHPPH",        # 6 residues
    "HPHPPHHP",      # 8 residues
    "HPHPPHHPHH",    # 10 residues
]

scaling_results = []

for seq in test_sequences:
    print(f"Testing sequence length {len(seq)}...")
    
    model = VQEFoldingModel(
        sequence=seq,
        lattice_dim=2,
        ansatz_depth=2,
        optimizer='COBYLA'
    )
    
    start = time.time()
    result = model.run(maxiter=50)
    elapsed = time.time() - start
    
    scaling_results.append({
        'length': len(seq),
        'n_qubits': model.encoding.n_qubits,
        'energy': result.optimal_value,
        'time': elapsed,
        'iterations': result.n_iterations
    })
    
    print(f"  Qubits: {model.encoding.n_qubits}, Time: {elapsed:.2f}s\n")

# Plot scaling
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

lengths = [r['length'] for r in scaling_results]
qubits = [r['n_qubits'] for r in scaling_results]
times = [r['time'] for r in scaling_results]

# Qubit scaling
axes[0].plot(lengths, qubits, 'o-', linewidth=2, markersize=10)
axes[0].set_xlabel('Protein Length', fontsize=12)
axes[0].set_ylabel('Number of Qubits', fontsize=12)
axes[0].set_title('Qubit Scaling', fontweight='bold', fontsize=13)
axes[0].grid(True, alpha=0.3)

# Time scaling
axes[1].plot(lengths, times, 's-', linewidth=2, markersize=10, color='orange')
axes[1].set_xlabel('Protein Length', fontsize=12)
axes[1].set_ylabel('Time (s)', fontsize=12)
axes[1].set_title('Time-to-Solution Scaling', fontweight='bold', fontsize=13)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/vqe_scaling.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nScaling Summary:")
for r in scaling_results:
    print(f"  N={r['length']:2d}: {r['n_qubits']:2d} qubits, {r['time']:6.2f}s")

## Summary

In this tutorial, we covered:

1. ✅ VQE theory and Hamiltonian construction
2. ✅ Ansatz design and circuit resources
3. ✅ Optimizer comparison (COBYLA, SLSQP, SPSA)
4. ✅ Parameter initialization strategies
5. ✅ Noise analysis framework
6. ✅ Convergence analysis
7. ✅ Scaling behavior with protein length

### Key Insights

- **Optimizer choice** matters: COBYLA is derivative-free and robust, SLSQP faster but needs gradients
- **Ansatz depth** trades expressivity for optimization difficulty
- **Initialization** can significantly affect convergence speed
- **Qubit count** scales roughly linearly with protein length (for turn encoding)
- **Time complexity** increases with both protein size and ansatz depth

### Next Steps

- Explore **03_qaoa_tutorial.ipynb** for QAOA-specific techniques
- Try **04_benchmarking.ipynb** for systematic comparison

---

**Questions?** Contact: marena@cua.edu